In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


# Update the script to include the TraditionalEEGModel class and run_traditional_ml_model function

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from scipy.signal import welch
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler



In [2]:
class TraditionalEEGModel:
    def __init__(self, k=3):
        self.k = k
        self.model = KNeighborsClassifier(n_neighbors=self.k)
        self.scaler = StandardScaler()

    def extract_features(self, signal, fs=256):
        # Use Welch’s method to calculate the power spectral density
        f, psd = welch(signal, fs=fs, nperseg=128)
        total_power = np.sum(psd)

        mean_freq = np.sum(f * psd) / total_power
        median_freq = f[np.argsort(np.cumsum(psd))[len(psd) // 2]]
        spectral_entropy = entropy(psd / total_power)
        spectral_centroid = np.sum(f * psd) / np.sum(psd)

        return [mean_freq, median_freq, spectral_entropy, spectral_centroid]

    def prepare_dataset(self, data, labels):
        feature_set = []
        for sample in data:
            trimmed = sample[934:]
            features = self.extract_features(trimmed)
            feature_set.append(features)
        feature_set = np.array(feature_set)
        return feature_set, np.array(labels)

    def fit(self, X, y):
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)

    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)









In [ ]:
# updated_code_svm = updated_code.replace(
#     "from sklearn.neighbors import KNeighborsClassifier",
#     "from sklearn.svm import SVC"
# ).replace(
#     "self.model = KNeighborsClassifier(n_neighbors=self.k)",
#     "self.model = SVC(kernel='linear', probability=True)"
# )




In [8]:
# Define paths
data_path = '/Users/yunhaoluo/Desktop/EEG_analysis/processed_data/'
train_features_path = data_path + "train.csv"
train_labels_path = data_path + "train_labels.csv"
test_features_path = data_path + "test.csv"
test_labels_path = data_path + "test_labels.csv"

X_train = pd.read_csv(train_features_path) 
X_test = pd.read_csv(test_features_path) 
y_train = pd.read_csv(train_labels_path) 
y_test = pd.read_csv(test_labels_path) 

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



In [11]:
# Train KNN classifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67       542
         1.0       0.94      0.06      0.11       571

    accuracy                           0.52      1113
   macro avg       0.72      0.53      0.39      1113
weighted avg       0.73      0.52      0.38      1113

AUC-ROC: 0.5279273107967507


/Users/yunhaoluo/anaconda3/envs/EEGDiff/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [12]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report





In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
svm = SVC(kernel='rbf', C=1.0)  # or try 'rbf', 'poly', etc.
svm.fit(X_train_scaled, y_train)

/Users/yunhaoluo/anaconda3/envs/EEGDiff/lib/python3.9/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC()

In [18]:
y_pred = svm.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.69      0.74      0.72       542
         1.0       0.74      0.69      0.71       571

    accuracy                           0.71      1113
   macro avg       0.72      0.71      0.71      1113
weighted avg       0.72      0.71      0.71      1113



In [19]:
print("AUC-ROC:", roc_auc_score(y_test, y_pred))

AUC-ROC: 0.7149349558294182
